In [ ]:
%pip install bertopic

: 

In [ ]:
%pip install sklearn

: 

In [ ]:
%pip install umap-learn

: 

In [4]:
%pip install pip install hdbscan

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from nltk.tokenize import sent_tokenize, word_tokenize

: 

In [181]:

# https://maartengr.github.io/BERTopic/algorithm/algorithm.html#1-embed-documents

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()



representation_model = KeyBERTInspired()





In [182]:
topic_model = BERTopic(
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model # Step 6 - (Optional) Fine-tune topic represenations
  # nr_topics="auto"
)



In [7]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all')['data']
# topic_model = BERTopic(nr_topics="auto")


In [8]:
topics, probabilities = topic_model.fit_transform(docs)

NameError: name 'topic_model' is not defined

In [ ]:
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)


In [8]:
topic_model = BERTopic.load("topic_model")

In [9]:
topic_model.visualize_topics()

In [147]:
# topic_model.save("topic_model")

In [148]:
# BERTopic.load("topic_model")

In [ ]:
def get_topic_number(text):
  topics = topic_model.find_topics(text, top_n=1)
  return topics[0][0]

In [ ]:
def get_representative_topic_words(text):
  topic_number = get_topic_number(text)
  topics = topic_model.get_topic(topic_number),topic_number
  topic_names = ','.join([topic for topic, freq in topics[0]][:4])
  return topic_names


In [ ]:
get_representative_topic_words("""I'm sorry, but "mtld" is not a recognizable term or acronym in English. Can you please provide more context or clarify what you are referring to?""")

'motif,unix,bindings'

In [196]:
# word = "I really want to go get some popcorn when we go to the movies later"
# word = "I wonder if we will be able to get to the movies on time to watch startreck?"
get_representative_topic_words("hello")

(   Topic  Count                            Name       CustomName
 0    193     18  193_42_tiff_tiff6_significance  42, tiff, tiff6,
 193)

In [189]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,
  topic_prefix=False,
  word_length=15,
  separator=", ")
topic_model.set_topic_labels(topic_labels)

In [190]:
# -1 is non relevant topics
topic_model.get_topic_info()

,Topic,Count,Name,CustomName
0,-1,6111,-1_think_information_david_article,"think, information, david"
1,0,888,0_pitchers_pitcher_batting_pitching,"pitchers, pitcher, batting"
2,1,483,1_encryption_crypto_cryptography_encrypted,"encryption, crypto, cryptography"
3,2,416,2_investigation_fbi_evidence_murder,"investigation, fbi, evidence"
4,3,407,3_firearm_firearms_handgun_guns,"firearm, firearms, handgun"
...,...,...,...,...
209,208,16,208_icons_iconize_icon_program,"icons, iconize, icon"
210,209,16,209_omnipotent_omnipotence_creates_existance,"omnipotent, omnipotence, creates"
211,210,16,210_tires_tire_tread_wheels,"tires, tire, tread"
212,211,15,211_orthodox_coptic_ecumenical_nestorians,"orthodox, coptic, ecumenical"


In [ ]:
topic_model.custom_labels_

In [259]:
def mtld(tokens, threshold=0.72):
    """
    Calculate the Measure of Textual Lexical Diversity (MTLD) score for a given list of tokens.
    Generates  groups to calculate the type-token ratio (TTR) determined by local diversity + threshold.
    Takes the average of the forward and backward to genreate the MTLD score.
    Args:
        tokens: A list of tokens (words) in the language sample.
        threshold: The minimum type-token ratio (TTR) required to maintain lexical diversity.
    Returns:
        The MTLD score for the language sample.
    """
    ttr, factor_count, word_count = 1.0, 1, len(tokens)
    stack = []
    for token in tokens:
        stack.append(token)
        ttr = len(set(stack)) / len(stack)
        if ttr < threshold:
            stack = []
            factor_count += 1
    mtld_score = word_count / factor_count
    reversed_tokens = list(reversed(tokens))
    ttr, factor_count, word_count = 1.0, 1, len(reversed_tokens)
    stack = []
    for token in reversed_tokens:
        stack.append(token)
        ttr = len(set(stack)) / len(stack)
        if ttr < threshold:
            stack = []
            factor_count += 1
    reversed_mtld_score = word_count / factor_count
    return (mtld_score + reversed_mtld_score) / 2

In [267]:
def mattr(tokens, window_size):
    """
    Calculate a lexical diversity score using a moving average of type-token ratios (MATTR).
    Args:
        tokens: A list of tokens (words) in the language sample.
        window_size: The size of the window used for calculating TTRs.
    Returns:
        The MATTR score for the language sample.
    """
    ttrs = []
    for i in range(len(tokens) - window_size + 1):
        window = tokens[i:i+window_size]
        ttr = len(set(window)) / window_size
        ttrs.append(ttr)
    mattr_score = sum(ttrs) / len(ttrs)
    return mattr_score

In [268]:
# https://github.com/jennafrens/lexical_diversity/blob/master/lexical_diversity.py
import string

# Global trandform for removing punctuation from words
remove_punctuation = str.maketrans('', '', string.punctuation)

# HD-D internals

# x! = x(x-1)(x-2)...(1)
def factorial(x):
    if x <= 1:
        return 1
    else:
        return x * factorial(x - 1)

# n choose r = n(n-1)(n-2)...(n-r+1)/(r!)
def combination(n, r):
    r_fact = factorial(r)
    numerator = 1.0
    num = n-r+1.0
    while num < n+1.0:
        numerator *= num
        num += 1.0
    return numerator / r_fact

# hypergeometric probability: the probability that an n-trial hypergeometric experiment results 
#  in exactly x successes, when the population consists of N items, k of which are classified as successes.
#  (here, population = N, population_successes = k, sample = n, sample_successes = x)
#  h(x; N, n, k) = [ kCx ] * [ N-kCn-x ] / [ NCn ]
def hypergeometric(population, population_successes, sample, sample_successes):
    return (combination(population_successes, sample_successes) *\
            combination(population - population_successes, sample - sample_successes)) /\
            combination(population, sample)
    
# HD-D implementation
def hdd(word_array, sample_size=42.0):
    if isinstance(word_array, str):
        raise ValueError("Input should be a list of strings, rather than a string. Try using string.split()")
    if len(word_array) < 50:
        raise ValueError("Input word list should be at least 50 in length")

    # Create a dictionary of counts for each type
    type_counts = {}
    for token in word_array:
        token = token.translate(remove_punctuation).lower() # trim punctuation, make lowercase
        if token in type_counts:
            type_counts[token] += 1.0
        else:
            type_counts[token] = 1.0
    # Sum the contribution of each token - "If the sample size is 42, the mean contribution of any given
    #  type is 1/42 multiplied by the percentage of combinations in which the type would be found." (McCarthy & Jarvis 2010)
    hdd_value = 0.0
    for token_type in type_counts.keys():
        contribution = (1.0 - hypergeometric(len(word_array), sample_size, type_counts[token_type], 0.0)) / sample_size
        hdd_value += contribution

    return hdd_value

In [269]:
import pandas as pd

ex_text = """"
re, here are five sentences on five random topics:

Coffee: Coffee is one of the most widely consumed beverages in the world, and it is enjoyed for its rich flavor and energizing effects. From a simple cup of black coffee to more complex creations like cappuccinos and lattes, coffee is a staple of many people's daily routines. However, excessive coffee consumption can lead to negative health effects like anxiety and insomnia, so it's important to consume in moderation.
Travel: Traveling allows us to experience new cultures, try new foods, and see the world from a different perspective. Whether it's backpacking through Europe or taking a road trip through the United States, travel can be a transformative experience that broadens our horizons and helps us appreciate the beauty and diversity of our planet.
Sports: Sports have the ability to bring people together and foster a sense of community and belonging. Whether it's playing on a team or watching a game with friends, sports can provide a source of enjoyment and camaraderie. However, it's important to remember that winning isn't everything and that sportsmanship and fair play are essential components of any game.
Music: Music has the power to evoke emotions and memories, transport us to different places and times, and connect us with other people. Whether it's listening to a favorite song or attending a live concert, music can be a source of joy and inspiration. Additionally, learning to play an instrument or sing can provide a creative outlet and help develop new skills.
Artificial Intelligence: Artificial Intelligence (AI) is an exciting field that has the potential to revolutionize the way we live and work. From self-driving cars to intelligent personal assistants like Siri and Alexa, AI technology is rapidly advancing and changing the way we interact with machines. However, there are also concerns about the potential negative impacts of AI, such as the displacement of jobs and the loss of privacy. It's important to continue to research and develop AI technology in a responsible and ethical manner.
"""


sent_ex_text = sent_tokenize(ex_text)

def text_to_diversity_per_topic(text, diversity_function):
  tokenized_text = sent_tokenize(text)
  window_to_sentence = {}
  for idx, i in enumerate(tokenized_text):
    topic_for_sentence = get_representative_topic_words(i)
    window_to_sentence[topic_for_sentence] = {*window_to_sentence.get(topic_for_sentence, {}), idx}
  diversity = {}
  total_diversity_scores = 0
  # calculate the diversity for each group
  for topic, sentence_indexes in window_to_sentence.items():
    bag_of_words = ' '.join([tokenized_text[idx] for idx in sentence_indexes])
    diversity_score = diversity_function(bag_of_words)
    diversity[topic] = diversity_score
    total_diversity_scores += diversity_score


  return diversity, total_diversity_scores/len(diversity)
text_to_diversity_per_topic(ex_text,mtld)

({'newsgroup,newsgroups,groups,split': 11.8125,
  'crohn,colitis,dietary,gastroenterology': 11.217948717948719,
  'obesity,dieting,rebound,diet': 12.545454545454547,
  'airfare,airfares,tickets,ticket': 12.592592592592593,
  'espn,sportschannel,espn2,televised': 11.875,
  'hypocrisy,hypocritical,hypocrite,jim': 11.748251748251748,
  'stereo,audio,soundbase,speakers': 11.97043010752688,
  'scientific,scientists,studies,scientist': 12.727272727272727,
  'interrupts,interrupt,irq7,ports': 13.416666666666666,
  'morality,morals,moral,ethics': 13.882352941176471},
 12.378847004689035)

{1, 2, 3, 4}